# Content Based Recommender: For Books 10k Dataset.

## We do three steps: 
### 1. Extract Features into weights
### 2. Train a multiway logisitic regression model
### 3. Recommend

In [138]:
import numpy as np
import pandas as pd

### We Need to import the books, ratings, and  tags for each book.


In [139]:
booksRaw = pd.read_csv('./datasets/goodBooks10k/books.csv')
ratingsRaw = pd.read_csv('./datasets/goodBooks10k/ratings.csv')
tagsRaw = pd.read_csv('./datasets/goodBooks10k/tags.csv')
bookTagsRaw = pd.read_csv('./datasets/goodBooks10k/book_tags.csv')
toReadRaw = pd.read_csv('./datasets/goodBooks10k/to_read.csv')
booksRaw.head()

,id,book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


In [140]:
booksRaw

,id,book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...
5,6,11870085,11870085,16827462,226,525478817,9.780525e+12,John Green,2012.0,The Fault in Our Stars,...,2346404,2478609,140739,47994,92723,327550,698471,1311871,https://images.gr-assets.com/books/1360206420m...,https://images.gr-assets.com/books/1360206420s...
6,7,5907,5907,1540236,969,618260307,9.780618e+12,J.R.R. Tolkien,1937.0,The Hobbit or There and Back Again,...,2071616,2196809,37653,46023,76784,288649,665635,1119718,https://images.gr-assets.com/books/1372847500m...,https://images.gr-assets.com/books/1372847500s...
7,8,5107,5107,3036731,360,316769177,9.780317e+12,J.D. Salinger,1951.0,The Catcher in the Rye,...,2044241,2120637,44920,109383,185520,455042,661516,709176,https://images.gr-assets.com/books/1398034300m...,https://images.gr-assets.com/books/1398034300s...
8,9,960,960,3338963,311,1416524797,9.781417e+12,Dan Brown,2000.0,Angels & Demons,...,2001311,2078754,25112,77841,145740,458429,716569,680175,https://images.gr-assets.com/books/1303390735m...,https://images.gr-assets.com/books/1303390735s...
9,10,1885,1885,3060926,3455,679783261,9.780680e+12,Jane Austen,1813.0,Pride and Prejudice,...,2035490,2191465,49152,54700,86485,284852,609755,1155673,https://images.gr-assets.com/books/1320399351m...,https://images.gr-assets.com/books/1320399351s...


In [141]:
# just merge the tags together!
tags_join_DF = pd.merge(bookTagsRaw, tagsRaw, left_on='tag_id', right_on='tag_id', how='inner')
tags_join_DF.head()

,goodreads_book_id,tag_id,count,tag_name
0,1,30574,167697,to-read
1,2,30574,24549,to-read
2,3,30574,496107,to-read
3,5,30574,11909,to-read
4,6,30574,298,to-read


In [156]:
titles = booksRaw['title']
indices = pd.Series(booksRaw.index, index = titles)
taggedBooks = pd.merge(booksRaw, tags_join_DF, left_on='book_id', right_on='goodreads_book_id', how='inner')
taggedBooks= taggedBooks.dropna()

In [157]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
#we use these two to convert text to possible features
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
bagOfWordsMatrix = tf.fit_transform(taggedBooks['tag_name'].head(10000))
cosSim = linear_kernel(bagOfWordsMatrix, bagOfWordsMatrix)

#sanity check
#x = [i for i in cosSim[1] if i>0]


In [197]:
# now lets be able to look at tag similiarity
def getTagPrediction(title):
    theBook = indices[title]
    sim_scores = list(enumerate(cosSim[theBook]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:21]
    book_indices = [i[0] for i in sim_scores]
    return booksRaw.iloc[book_indices]['title']

print(taggedBooks[taggedBooks['title']=="The Great Gatsby"]['tag_name'][:20])
print(getTagPrediction('Naked Lunch'))

400               to-read
401             favorites
402     currently-reading
403           young-adult
404               fiction
405           books-i-own
406                 owned
407            favourites
408           owned-books
409               re-read
410    all-time-favorites
411               default
412              my-books
413              classics
414                reread
415                 i-own
416             audiobook
417               5-stars
418        favorite-books
419                novels
Name: tag_name, dtype: object
1570                                          Naked Lunch
2169       Smoke and Mirrors: Short Fiction and Illusions
3077                               The Paper Bag Princess
3280                     The Iron Trial (Magisterium, #1)
3473                      Tower Lord (Raven's Shadow, #2)
3572                          Burning Chrome (Sprawl, #0)
3780                  The Goddess Test (Goddess Test, #1)
3975                                        

In [159]:
uniqueUsers = ratingsRaw['user_id'].drop_duplicates()
taggedBooks.head()
ratingsRaw

,book_id,user_id,rating
0,1,314,5
1,1,439,3
2,1,588,5
3,1,1169,4
4,1,1185,4
5,1,2077,4
6,1,2487,4
7,1,2900,5
8,1,3662,4
9,1,3922,5


# We're going to set up a user profile now. We're going to make a function where you input a unique user, and we will get the unique books and their rating for each book. From there, we will train a model to fit that user best. 

In [194]:
def makeUserProfileData(userID): #user ID must be int
    result = ratingsRaw[ratingsRaw['user_id']==9246]
        
    highStarID = result[result['rating']>=4]['book_id'].values
    lowStarID = result[result['rating']<=2]['book_id'].values
    highStarTags = [taggedBooks[taggedBooks['id'] == i]['tag_name'] for i in highStarID]
    lowStarTags = [taggedBooks[taggedBooks['id'] == i]['tag_name'] for i in lowStarID]
    


    for i in highStarTags:
        print(i)
        i=i.set_value(max(i.index)+1,'high')
    for j in lowStarTags:
        j=j.set_value(max(j.index)+1,'low')
    columns = (highStarID.tolist())+ (lowStarID.tolist())
    data= highStarTags+lowStarTags
    #final
    bookTag = pd.DataFrame(columns=columns)
    count = 0
    for i in rip.columns:
        if count < len(rip.columns):
            rip[i] = data[count].values[1:]
            #print('count: ',count,'\n',data[count])
            count+=1
    #rip.iloc[len(rip)] = 
    return rip
makeUserProfileData(9246)

#fives= (x[x['rating']==5]['book_id'])

#taggedBooks[taggedBooks['id']==fives.values[3]]['tag_name']

#taggedBooks[taggedBooks['book_id']==fives.values]

#x['fiveStar'] = x[x['rating' == 5]]

#ratingsRaw[ratingsRaw['user_id']==48386]

300                to-read
301              favorites
302      currently-reading
303            young-adult
304                fiction
305                  owned
306                     ya
307             favourites
308            owned-books
309                re-read
310              childhood
311     all-time-favorites
312                default
313               my-books
314               classics
315                 reread
316                  i-own
317              audiobook
318                5-stars
319         favorite-books
320                 novels
321               favorite
322             audiobooks
323             my-library
324                english
325                  books
326           read-in-2016
327                mystery
328           my-favorites
329                 own-it
              ...         
370        read-for-school
371             for-school
372           school-books
373         read-in-school
374           school-reads
375        general-fiction
3

/home/robot-tumas/pyEnv/wiggles/lib/python3.5/site-packages/ipykernel_launcher.py:13: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  del sys.path[0]


700                   to-read
701                 favorites
702         currently-reading
703               young-adult
704                   fiction
705               books-i-own
706                     owned
707                        ya
708                favourites
709               owned-books
710                   re-read
711        all-time-favorites
712                   default
713                  my-books
714                  classics
715                    reread
716                     i-own
717            favorite-books
718                    novels
719                  favorite
720       read-more-than-once
721                my-library
722                      teen
723                   english
724                     books
725              read-in-2016
726              my-favorites
727                    own-it
728                   library
729                     novel
                ...          
770                for-school
771              school-books
772       

3000                  to-read
3001                favorites
3002        currently-reading
3003              young-adult
3004                  fiction
3005              books-i-own
3006                    owned
3007               favourites
3008              owned-books
3009       all-time-favorites
3010                  default
3011                 my-books
3012                 classics
3013                    i-own
3014                audiobook
3015                  5-stars
3016           favorite-books
3017                   novels
3018                 favorite
3019               audiobooks
3020               my-library
3021                  english
3022                    books
3023             my-favorites
3024                   own-it
3025                  library
3026                    audio
3027                    novel
3028                    faves
3029       shelfari-favorites
                ...          
3070       historical-fiction
3071         african-american
3072      

6000                   to-read
6001                 favorites
6002                   fiction
6003               books-i-own
6004                     owned
6005                favourites
6006               owned-books
6007                   default
6008                  my-books
6009                     i-own
6010                 audiobook
6011                    novels
6012                audiobooks
6013                my-library
6014                   english
6015                   british
6016                     books
6017              read-in-2016
6018                   mystery
6019                    own-it
6020                   library
6021                     audio
6022                     novel
6023              read-in-2015
6024                    kindle
6025                    to-buy
6026                     ebook
6027              contemporary
6028           read-in-english
6029              read-in-2017
                 ...          
6070               stand-alone
6071    

12300                   to-read
12301                 favorites
12302         currently-reading
12303               young-adult
12304                   fiction
12305               books-i-own
12306                     owned
12307                favourites
12308               owned-books
12309        all-time-favorites
12310                   default
12311                  my-books
12312                     i-own
12313                 audiobook
12314                   5-stars
12315            favorite-books
12316                    novels
12317                audiobooks
12318                my-library
12319                   english
12320              read-in-2016
12321                   mystery
12322                    own-it
12323                   library
12324                     audio
12325                     novel
12326              read-in-2015
12327        shelfari-favorites
12328                    kindle
12329                    to-buy
                  ...          
12370   

Series([], Name: tag_name, dtype: object)


ValueError: max() arg is an empty sequence

In [167]:
result = ratingsRaw[ratingsRaw['user_id']==9246]

highStarID = result[result['rating']>=4]['book_id'].values
lowStarID = result[result['rating']<=2]['book_id'].values
highStarTags = [taggedBooks[taggedBooks['id'] == i]['tag_name'] for i in highStarID]
lowStarTags = [taggedBooks[taggedBooks['id'] == i]['tag_name'] for i in lowStarID]
columns = (highStarID.tolist())+ (lowStarID.tolist())
data= highStarTags+lowStarTags
#final
bookTag = pd.DataFrame(columns=columns)
count = 0
#for i in rip.columns:
#    if count < len(rip.columns):
##        rip[i] = data[count].values[1:]
    #print('count: ',count,'\n',data[count])
#        count+=1
#rip
#len(lowStarID)

In [181]:
highStarID[23]

143

In [193]:
(taggedBooks['id'].unique()==143)

False